# Fair CG Experiments
This notebook includess the experiments we used to test the effectiveness of our Fair CG algorithm. We start with a fixed rule set for each fold/dataset (generate in Far CG Rule Generation) and then run the fair cg algorithm starting from that ruleset.

In [ ]:
import pandas as pd
from test_helpers import *

In [ ]:
results = []
datasets = ['adult','compas','default']
protected_features = {'compas': 'race', 
                      'adult': 'gender',
                      'default': 'X2'
                     }
fair_met = 'EqOfOp'
test_params = {
            'price_limit': 45,
            'train_limit': 300,
            'fixed_model_params': {
                'ruleGenerator': 'Hybrid',
                'masterSolver':'barrierCrossover',
                'numRulesToReturn': 100,
                'fairness_module': 'EqOfOp'
            },
        }

eps_range = {'adult': [0, 0.01, 0.05, 0.1, 1],
    'compas':[0, 0.01, 0.05, 0.1, 0.15, 0.2, 1],
     'default': [0, 0.01, 0.05, 0.1, 1]
    }
C_range = {'adult': [40, 5, 15, 30],
    'compas': [5, 10, 20, 30],
     'default': [5, 10, 20, 30]
    }
epsilons = [0, 0.01, 0.05, 0.1, 0.5, 1]
test_complex = [10, 15, 20]

for i in range(10):
    print('***** FOLD %d ******'%i)

    for dataset in datasets:
        print('**** DATASET %s ******'%dataset)
        group_var = protected_features[dataset]
        name = dataset+'_'+str(fair_met)

        train  = pd.read_csv('data_split/bin_'+dataset+'_train_%d.csv'%i)
        test = pd.read_csv('data_split/bin_'+dataset+'_test_%d.csv'%i)
        
        isFirst = True
        for ruleset in ['faircgEqOp','faircgEqOd','rf','dt']:
            print('***** Ruleset %s ******'%ruleset)
            new_rules = np.load('rules/'+ruleset+'_%s_fold_%d.npy'%(dataset, i)).astype(int)
            if isFirst:
                saved_rules = new_rules
                isFirst = False
            else:
                saved_rules = np.concatenate([saved_rules,new_rules])

        for eps in eps_range[dataset]:
            for C in C_range[dataset]:
                res = TestResults(name+' '+'(%d,%d)_final'%(eps, C)+'-'+str(i))
                res.res['eps'] = eps
                res.res['C'] = C

                test_params = test_params.copy()
                test_params['fixed_model_params']['epsilon'] = eps
                test_params['fixed_model_params']['ruleComplexity'] = C
                
                print(test_params)
                res, classif = runSingleTest(train.drop('Y',axis=1).to_numpy(), train['Y'].to_numpy(), 
                                             train[group_var].to_numpy(), 
                                             test.drop('Y',axis=1).to_numpy(), test['Y'].to_numpy(), 
                                             test[group_var].to_numpy(), 
                                             test_params, 
                                             saved_rules, res, colGen = True, rule_filter = True)
                
                rules = classif.ruleMod.rules
                saved_rules = updateRuleSet(saved_rules,rules)

In [ ]:
results = []
datasets = ['adult','compas','default']
protected_features = {'compas': 'race', 
                      'adult': 'gender',
                      'default': 'X2'
                     }
fair_met = 'EqOfOp'
test_params = {
            'price_limit': 45,
            'train_limit': 300,
            'fixed_model_params': {
                'ruleGenerator': 'Hybrid',
                'masterSolver':'barrierCrossover',
                'numRulesToReturn': 100,
                'fairness_module': 'HammingEqOdd'
            },
        }

eps_range = {'adult': [0, 0.01, 0.05, 0.1, 1],
    'compas':[0, 0.01, 0.05, 0.1, 0.15, 0.2, 1],
     'default': [0, 0.01, 0.05, 0.1, 1]
    }
C_range = {'adult': [40, 5, 15, 30],
    'compas': [5, 10, 20, 30],
     'default': [5, 10, 20, 30]
    }
epsilons = [0, 0.01, 0.05, 0.1, 0.5, 1]
test_complex = [10, 15, 20]

for i in range(10):
    print('***** FOLD %d ******'%i)

    for dataset in datasets:
        print('**** DATASET %s ******'%dataset)
        group_var = protected_features[dataset]
        name = dataset+'_'+str(fair_met)

        train  = pd.read_csv('data_split/bin_'+dataset+'_train_%d.csv'%i)
        test = pd.read_csv('data_split/bin_'+dataset+'_test_%d.csv'%i)
        
        isFirst = True
        for ruleset in ['faircgEqOp','faircgEqOd','rf','dt']:
            print('***** Ruleset %s ******'%ruleset)
            new_rules = np.load('rules/'+ruleset+'_%s_fold_%d.npy'%(dataset, i)).astype(int)
            if isFirst:
                saved_rules = new_rules
                isFirst = False
            else:
                saved_rules = np.concatenate([saved_rules,new_rules])

        for eps in eps_range[dataset]:
            for C in C_range[dataset]:
                res = TestResults(name+' '+'(%d,%d)_final'%(eps, C)+'-'+str(i))
                res.res['eps'] = eps*2
                res.res['C'] = C

                test_params = test_params.copy()
                test_params['fixed_model_params']['epsilon'] = eps*2
                test_params['fixed_model_params']['ruleComplexity'] = C
                
                print(test_params)
                res, classif = runSingleTest(train.drop('Y',axis=1).to_numpy(), train['Y'].to_numpy(), 
                                             train[group_var].to_numpy(), 
                                             test.drop('Y',axis=1).to_numpy(), test['Y'].to_numpy(), 
                                             test[group_var].to_numpy(), 
                                             test_params, 
                                             saved_rules, res, colGen = True, rule_filter = True)
                
                rules = classif.ruleMod.rules
                saved_rules = updateRuleSet(saved_rules,rules)